This code was heavily based on Enge et al. 2021 codes available at https://osf.io/34ry2/ (we are highly grateful for their commitment to open research)

In [4]:
import random
from glob import glob
from math import sqrt
from os import makedirs, path
from re import sub
from shutil import copy
from sys import argv

import numpy as np
import pandas as pd
from nibabel import save
from nilearn import image, reporting
from nimare import correct, io, meta, utils
from scipy.stats import norm

In [5]:
def generate_null(
    text_file="peaks.txt",
    space="ale_2mm",
    k_null=100,
    random_seed=None,
    output_dir="./",
):

    # Load NiMARE's gray matter template
    temp = utils.get_template(space=space, mask="brain")

    # Extract possible MNI coordinates for all gray matter voxels
    x, y, z = np.where(temp.get_fdata() == 1.0)
    within_mni = image.coord_transform(x=x, y=y, z=z, affine=temp.affine)
    within_mni = np.array(within_mni).transpose()

    # Read the original Sleuth file into a NiMARE data set
    dset = io.convert_sleuth_to_dataset(text_file, target=space)

    # Set a random seed to make the results reproducible
    if random_seed:
        random.seed(random_seed)

    # Resample numbers of subjects per experiment based on the original data
    nr_subjects_dset = [n[0] for n in dset.metadata["sample_sizes"]]
    nr_subjects_null = random.choices(nr_subjects_dset, k=k_null)

    # Resample numbers of peaks per experiment based on the original data
    nr_peaks_dset = dset.coordinates["study_id"].value_counts().tolist()
    nr_peaks_null = random.choices(nr_peaks_dset, k=k_null)

    # Create random peak coordinates
    idx_list = [
        random.sample(range(len(within_mni)), k=k_peaks) for k_peaks in nr_peaks_null
    ]
    peaks_null = [within_mni[idx] for idx in idx_list]

    # Copy original experiments to the destination Sleuth file
    makedirs(output_dir, exist_ok=True)
    text_file_basename = path.basename(text_file)
    null_file_basename = sub(
        pattern=".txt", repl="_plus_k" + str(k_null) + ".txt", string=text_file_basename
    )
    null_file = output_dir + "/" + null_file_basename
    copy(text_file, null_file)

    # Append all the null studies to the Sleuth file
    f = open(null_file, mode="a")
    for i in range(k_null):
        f.write(
            "\n// nullstudy"
            + str(i + 1)
            + "\n// Subjects="
            + str(nr_subjects_null[i])
            + "\n"
        )
        np.savetxt(f, peaks_null[i], fmt="%.3f", delimiter="\t")
    f.close()

    # Read the new Sleuth file and return it as a NiMARE data set
    dset_null = io.convert_sleuth_to_dataset(null_file, target=space)
    return dset_null

In [7]:
text_file = "/Users/ss/Documents/Self_Psych_Meta/Data/AnalysisData/Coordinates/control_minus_patient.txt"

In [8]:
# Define function to compute the FSN for all voxels from a Sleuth file
def compute_fsn(
    text_file="peaks.txt",
    space="ale_2mm",
    voxel_thresh=0.001,
    cluster_thresh=0.01,
    n_iters=1000,
    k_max_factor=5,
    random_ale_seed=None,
    random_null_seed=None,
    output_dir="./",
):

    # Let's show the user what we are doing
    print("\nCOMPUTING FSN FOR " + text_file + " (seed: " + str(random_null_seed) + ")")

    # Set random seed for original ALE if requested
    if random_ale_seed:
        np.random.seed(random_ale_seed)

    # Recreate the original ALE analysis
    ale = meta.cbma.ALE()
    corr = correct.FWECorrector(
        method="montecarlo", voxel_thresh=voxel_thresh, n_iters=n_iters
    )
    dset_orig = io.convert_sleuth_to_dataset(text_file=text_file, target=space)
    res_orig = ale.fit(dset_orig)
    cres_orig = corr.transform(res_orig)

    # Extract the original study IDs
    ids_orig = dset_orig.ids.tolist()

    # Create a new data set with a large number null studies added
    k_max = len(ids_orig) * k_max_factor
    dset_null = generate_null(
        text_file=text_file,
        space=space,
        k_null=k_max,
        random_seed=random_null_seed,
        output_dir=output_dir,
    )

    # Create thresholded cluster mask
    img_fsn = cres_orig.get_map("z_desc-size_level-cluster_corr-FWE_method-montecarlo")
    cluster_thresh_z = norm.ppf(1 - cluster_thresh / 2)
    img_fsn = image.threshold_img(img_fsn, threshold=cluster_thresh_z)
    img_fsn = image.math_img("np.where(img > 0, 1, 0)", img=img_fsn)

    # Create cluster-thresholded z map
    img_z = cres_orig.get_map("z")
    img_z = image.math_img("img1 * img2", img1=img_fsn, img2=img_z)

    # Iteratively add null studies up to our pre-defined maximum
    for k in range(1, k_max):

        # Print message
        print("Computing ALE for k = " + str(k) + " null studies added...")

        # Create a new data set with k null studies added
        ids_null = ["nullstudy" + str(x) + "-" for x in range(1, k + 1)]
        ids = ids_orig + ids_null
        dset_k = dset_null.slice(ids)

        # Compute the ALE
        res_k = res = ale.fit(dset_k)
        cres_k = corr.transform(result=res_k)

        # Create a thresholded cluster mask
        img_k = cres_k.get_map("z_desc-size_level-cluster_corr-FWE_method-montecarlo")
        img_k = image.threshold_img(img_k, threshold=cluster_thresh_z)
        img_k = image.math_img("np.where(img > 0, 1, 0)", img=img_k)

        # Use this to update the per-voxel FSN - this is a bit hack-ish: On a voxel-by-
        # voxel basis, we increase the value by 1 if and only if the voxel has remained
        # significant. As soon as it has failed to reach significance once, we never
        # increase FSN any further. This is handeled by comparing the current FSN to
        # the current value of k.
        count = str(k + 1)
        formula = "np.where(img_fsn + img_k == " + count + ", img_fsn + 1, img_fsn)"
        img_fsn = image.math_img(formula, img_fsn=img_fsn, img_k=img_k)

        # Quit as soon as there are no significant clusters left in the current map
        if not np.any(img_k.get_fdata()):
            print("No more significant voxels - terminating\n")
            break

    # Save the FSN map that we've created in the loop
    filename_img = path.basename(text_file).replace(".txt", "_fsn.nii.gz")
    save(img_fsn, filename=output_dir + "/" + filename_img)

    # Extract the FSN values at the original cluster peaks
    tab_fsn = reporting.get_clusters_table(img_z, stat_threshold=0, min_distance=1000)
    inv_affine = np.linalg.inv(img_z.affine)
    x, y, z = [np.array(tab_fsn[col]) for col in ["X", "Y", "Z"]]
    x, y, z = image.coord_transform(x=x, y=y, z=z, affine=inv_affine)
    x, y, z = [arr.astype("int") for arr in [x, y, z]]
    tab_fsn["FSN"] = img_fsn.get_fdata()[x, y, z]

    # Save this cluster table with the new FSN column
    filename_tab = path.basename(text_file).replace(".txt", "_fsn.tsv")
    tab_fsn.to_csv(output_dir + "/" + filename_tab, sep="\t", index=False)

    return img_fsn, tab_fsn

In [9]:
prefix = "control_minus_patient"  # 用于输出文件命名
output_dir_base = f"./results/fsn/{prefix}/"

In [10]:
nr_filedrawers = 5

In [11]:
random_null_seeds = random.sample(range(1000), k=nr_filedrawers)
filedrawers = [f"filedrawer{seed}" for seed in random_null_seeds]


In [12]:
for random_null_seed, filedrawer in zip(random_null_seeds, filedrawers):
    compute_fsn(
        text_file=text_file,
        space="ale_2mm",
        voxel_thresh=0.001,
        cluster_thresh=0.01,
        n_iters=1000,
        k_max_factor=5,
        random_ale_seed=1234,
        random_null_seed=random_null_seed,
        output_dir=output_dir_base + filedrawer,
    )


COMPUTING FSN FOR /Users/ss/Documents/Self_Psych_Meta/Data/AnalysisData/Coordinates/control_minus_patient.txt (seed: 434)


INFO:nimare.correct:Using correction method implemented in Estimator: nimare.meta.cbma.ale.ALE.correct_fwe_montecarlo.
100%|██████████| 1000/1000 [04:06<00:00,  4.05it/s]
INFO:nimare.meta.cbma.base:Using null distribution for voxel-level FWE correction.
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_21133/1937838368.py:46: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  img_fsn = image.threshold_img(img_fsn, threshold=cluster_thresh_z)
/Users/ss/miniconda3/lib/python3.12/site-packages/nilearn/image/image.py:1160: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the

Computing ALE for k = 1 null studies added...


INFO:nimare.correct:Using correction method implemented in Estimator: nimare.meta.cbma.ale.ALE.correct_fwe_montecarlo.
100%|██████████| 1000/1000 [05:00<00:00,  3.33it/s]
INFO:nimare.meta.cbma.base:Using null distribution for voxel-level FWE correction.
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_21133/1937838368.py:70: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  img_k = image.threshold_img(img_k, threshold=cluster_thresh_z)
/Users/ss/miniconda3/lib/python3.12/site-packages/nilearn/image/image.py:1160: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the dat

Computing ALE for k = 2 null studies added...


...

Computing ALE for k = 3 null studies added...


INFO:nimare.correct:Using correction method implemented in Estimator: nimare.meta.cbma.ale.ALE.correct_fwe_montecarlo.
100%|██████████| 1000/1000 [05:31<00:00,  3.02it/s]
INFO:nimare.meta.cbma.base:Using null distribution for voxel-level FWE correction.
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_21133/1937838368.py:70: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  img_k = image.threshold_img(img_k, threshold=cluster_thresh_z)
/Users/ss/miniconda3/lib/python3.12/site-packages/nilearn/image/image.py:1160: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the dat

Computing ALE for k = 4 null studies added...


INFO:nimare.correct:Using correction method implemented in Estimator: nimare.meta.cbma.ale.ALE.correct_fwe_montecarlo.
100%|██████████| 1000/1000 [05:18<00:00,  3.14it/s]
INFO:nimare.meta.cbma.base:Using null distribution for voxel-level FWE correction.
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_21133/1937838368.py:70: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  img_k = image.threshold_img(img_k, threshold=cluster_thresh_z)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_21133/1937838368.py:70: UserWarning: The given float value must not exceed 2.457263390205437. But, you have given threshold=2.5758293035489004.
  img_k = image.threshold_img(img_k, threshold=cluster_thresh_z)
/Users/ss/miniconda3/lib/python3.12/site-packages/nilearn/image/image

No more significant voxels - terminating



In [13]:
from glob import glob
import numpy as np
import pandas as pd
from nilearn import image
from nibabel import save
from scipy.stats import norm
from math import sqrt
import os


In [14]:
output_dir_base=f"./results/fsn/{prefix}/"
pattern = os.path.join(output_dir_base, "filedrawer*", f"{prefix}_fsn.nii.gz")
fnames_maps = glob(pattern)



In [15]:
prefix = "control_minus_patient"

In [17]:
# 读取所有 filedrawer 的 FSN 图像文件
fnames_maps = glob("../results/fsn/" + prefix + "/filedrawer*/" + prefix + "_fsn.nii.gz")
imgs_fsn = [image.load_img(fname) for fname in fnames_maps]

# 平均并保存 mean FSN 图像
img_mean = image.mean_img(imgs_fsn)
fname_img_mean = "../results/fsn/" + prefix + "/" + prefix + "_mean_fsn.nii.gz"
save(img_mean, fname_img_mean)

# 读取所有 filedrawer 的 FSN 表格
fnames_tabs = glob("../results/fsn/" + prefix + "/filedrawer*/" + prefix + "_fsn.tsv")
tabs_fsn = [pd.read_csv(fname, delimiter="\t") for fname in fnames_tabs]
tab_fsn = pd.concat(tabs_fsn)

# 计算统计指标
agg = tab_fsn.groupby("Cluster ID")["FSN"].agg(["mean", "count", "std"])
ci_level = 0.05
z_crit = abs(norm.ppf(ci_level / 2))
agg["se"] = agg["std"] / np.sqrt(agg["count"])
agg["ci_lower"] = agg["mean"] - z_crit * agg["se"]
agg["ci_upper"] = agg["mean"] + z_crit * agg["se"]

# 保存聚合结果为 CSV
fname_agg = "../results/fsn/" + prefix + "/" + prefix + "_mean_fsn.csv"
agg.to_csv(fname_agg, float_format="%.3f")

/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_21133/2391959793.py:6: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  img_mean = image.mean_img(imgs_fsn)
